In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pip install pulp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 52.2 MB/s eta 0:00:00


In [6]:
pip install gamspy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.

In [8]:
pip install numpy

In [7]:
# Tao data

import numpy as np

# Bien yeu cau
n = 8  # Số san pham
m = 5  # Số linh kien
S = 2  # Số scenarios

# Set seed de kem soat data
np.random.seed(0)

# Ma tran A
A = np.random.randint(1, 10, size=(n, m))

# b, l, q, s
b = np.random.uniform(1, 5, size=m)  # gia dat hang truoc/unit linh kien
l = np.random.uniform(5, 10, size=n)  # chi phi thoa man 1 unit demand cho moi san pham
q = np.random.uniform(1, 5, size=n)  # gia ban/1 san pham
s = np.random.uniform(0.5, 3, size=m)  # gia tri thu hoi/unit linh kien

#in data
A, b, l, q, s



(array([[6, 1, 4, 4, 8],
        [4, 6, 3, 5, 8],
        [7, 9, 9, 2, 7],
        [8, 8, 9, 2, 6],
        [9, 5, 4, 1, 4],
        [6, 1, 3, 4, 9],
        [2, 4, 4, 4, 8],
        [1, 2, 1, 5, 8]]),
 array([2.05822245, 4.09693476, 2.82460133, 3.2737358 , 1.0751592 ]),
 array([8.08817749, 8.06047861, 8.08466998, 9.71874039, 8.4091015 ,
        6.7975395 , 7.18515977, 8.48815598]),
 array([1.24090189, 3.66706686, 3.68255148, 1.84153024, 1.51570519,
        2.2617134 , 2.45484308, 3.28078708]),
 array([1.59650378, 2.9709346 , 0.75511203, 1.02219189, 0.90327379]))

In [9]:
# data demand

# tao data demand D cho tung san pham, theo  Bin(10, 0,5)
D = [np.random.binomial(10, 0.5, S) for _ in range(n)]

# in data
D



[array([6, 4]),
 array([5, 4]),
 array([3, 3]),
 array([6, 3]),
 array([4, 4]),
 array([6, 3]),
 array([7, 3]),
 array([8, 5])]

In [10]:
# set model

import pulp as lp

# tao model, Lp.LpProblem = linear programming problem, import pulp để giải, model này xác định mục tiêu là minimize
model = lp.LpProblem("2-SLPWR", lp.LpMinimize)

# decision variables
# xj: tạo xj với giá trị từ 0 -> m
x = lp.LpVariable.dicts("x", range(m), lowBound=0)

# y^kj: tương tự
y = [[lp.LpVariable(f"y_{k}_{j}", lowBound=0) for j in range(m)] for k in range(S)]

# z^ki: tương tự
z = [[lp.LpVariable(f"z_{k}_{i}", lowBound=0, upBound=10) for i in range(n)] for k in range(S)]

# ham muc tieu
# giam thieu tong chi phi bao gồm preorder cost và expected cost với các scenarios khac nhau
model += lp.lpSum(b[j] * x[j] for j in range(m)) + \
         lp.lpSum(0.5 * ((lp.lpSum((l[i] - q[i]) * z[k][i] for i in range(n))) - \
                                   lp.lpSum(s[j] * y[k][j] for j in range(m))) for k in range(S))

# gioi han
for k in range(S):
    for j in range(m):
        model += y[k][j] == x[j] - lp.lpSum(A[i][j] * z[k][i] for i in range(n))

    for i in range(n):
        model += z[k][i] <= D[i][k]


#in model
model

2-SLPWR:
MINIMIZE
2.058222448418508*x_0 + 4.096934757736866*x_1 + 2.824601328866194*x_2 + 3.273735795474594*x_3 + 1.0751592017454206*x_4 + -0.7982518918279005*y_0_0 + -1.4854672975740328*y_0_1 + -0.3775560134350351*y_0_2 + -0.5110959451185434*y_0_3 + -0.45163689735624535*y_0_4 + -0.7982518918279005*y_1_0 + -1.4854672975740328*y_1_1 + -0.3775560134350351*y_1_2 + -0.5110959451185434*y_1_3 + -0.45163689735624535*y_1_4 + 3.4236377994311535*z_0_0 + 2.196705875914718*z_0_1 + 2.2010592529505733*z_0_2 + 3.938605074138879*z_0_3 + 3.446698152449002*z_0_4 + 2.267913049586097*z_0_5 + 2.365158342613108*z_0_6 + 2.603684448982403*z_0_7 + 3.4236377994311535*z_1_0 + 2.196705875914718*z_1_1 + 2.2010592529505733*z_1_2 + 3.938605074138879*z_1_3 + 3.446698152449002*z_1_4 + 2.267913049586097*z_1_5 + 2.365158342613108*z_1_6 + 2.603684448982403*z_1_7 + 0.0
SUBJECT TO
_C1: - x_0 + y_0_0 + 6 z_0_0 + 4 z_0_1 + 7 z_0_2 + 8 z_0_3 + 9 z_0_4 + 6 z_0_5
 + 2 z_0_6 + z_0_7 = 0

_C2: - x_1 + y_0_1 + z_0_0 + 6 z_0_1 + 9 

In [11]:
# giai model
model.solve()

# tim ra gia tri x, y, z tối ưu
x_solution = {j: x[j].varValue for j in range(m)}
y_solution = {(k, j): y[k][j].varValue for k in range(S) for j in range(m)}
z_solution = {(k, i): z[k][i].varValue for k in range(S) for i in range(n)}

# In ket qua
print("Optimal values x", x_solution)
print("Optimal values y", y_solution)
print("Optimal values z", z_solution)



Optimal values x {0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4: -0.0}
Optimal values y {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0}
Optimal values z {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (0, 5): 0.0, (0, 6): 0.0, (0, 7): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (1, 5): 0.0, (1, 6): 0.0, (1, 7): 0.0}


In [ ]:
!zip -r /content/file.zip /content/Folder_To_Zip